In [23]:
import pandas as pd
import re
import os
import logging
from datetime import datetime

logging.basicConfig(filename='flood_data_extraction.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

def clean_text(text):
    return re.sub(r'\s+', ' ', text).strip()

def extract_date(filename):
    match = re.search(r'(\d{2}\.\d{2}\.\d{4})', filename)
    if match:
        return datetime.strptime(match.group(1), '%d.%m.%Y').strftime('%Y-%m-%d')
    return None

def parse_complex_cell(cell):
    if isinstance(cell, str) and '|' in cell:
        parts = re.findall(r'\(([^|]+)\s*\|\s*(\d+)\)', cell)
        return {name.strip(): int(count) for name, count in parts}
    return cell

In [24]:
def process_csv(csv_file):
    try:
        df = pd.read_csv(csv_file, header=None, skip_blank_lines=True)
    except FileNotFoundError:
        logging.error(f"CSV file not found: {csv_file}")
        return None
    except pd.errors.EmptyDataError:
        logging.error(f"CSV file is empty: {csv_file}")
        return None
    except Exception as e:
        logging.error(f"Error reading CSV file {csv_file}: {str(e)}")
        return None

    data = {}
    current_section = None

    try:
        for _, row in df.iterrows():
            if pd.notna(row[0]) and isinstance(row[0], str) and not row[0].isdigit():
                current_section = clean_text(row[0])
                data[current_section] = []
            elif current_section and pd.notna(row[1]):
                cleaned_row = [clean_text(str(cell)) if pd.notna(cell) else '' for cell in row]
                parsed_row = [parse_complex_cell(cell) for cell in cleaned_row]
                data[current_section].append(parsed_row)
    except Exception as e:
        logging.error(f"Error processing data in {csv_file}: {str(e)}")
        return None

    logging.info(f"Successfully processed CSV file: {csv_file}")
    return data

In [25]:
def extract_bridges_damaged(data):
    bridges = data.get('Infrastructure Damaged - Bridge', [])
    if not bridges:
        return pd.DataFrame()
    headers = ['District', 'Number', 'Revenue Circle', 'Bridge Name', 'Department', 'Village', 'Location', 'Longitude', 'Latitude', 'Remarks']
    df = pd.DataFrame(bridges[1:], columns=headers)
    return df

def extract_embankment_affected(data):
    embankments = data.get('Infrastructure Damaged - Embankment Affected', [])
    if not embankments:
        return pd.DataFrame()
    headers = ['District', 'Number', 'Revenue Circle', 'Embankment Affected Name', 'Department', 'Village', 'Location', 'Longitude', 'Latitude', 'Remarks']
    df = pd.DataFrame(embankments[1:], columns=headers)
    return df

def extract_embankment_breached(data):
    embankments = data.get('Infrastructure Damaged - Embankment Breached', [])
    if not embankments:
        return pd.DataFrame()
    headers = ['District', 'Number', 'Revenue Circle', 'Embankment Breached Name', 'Department', 'Village', 'Location', 'Longitude', 'Latitude', 'Remarks']
    df = pd.DataFrame(embankments[1:], columns=headers)
    return df

def extract_roads_damaged(data):
    roads = data.get('Infrastructure Damaged - Road', [])
    if not roads:
        return pd.DataFrame()
    headers = ['District', 'Number', 'Revenue Circle', 'Road Name', 'Department', 'Village', 'Location', 'Longitude', 'Latitude', 'Remarks']
    df = pd.DataFrame(roads[1:], columns=headers)
    return df

def extract_human_life_lost(data):
    lives_lost = data.get('Human Lives Lost - Confirmed', [])
    if not lives_lost:
        return pd.DataFrame()
    headers = ['District', 'Total', 'Flood Death', 'General Drowning (Non Flood)', 'Male', 'Female', 'Children Male', 'Children Female', 'Others', 'Revenue Circlewise']
    df = pd.DataFrame(lives_lost[1:], columns=headers)
    return df

def extract_population_affected(data):
    population = data.get('Population And Crop Area Affected', [])
    if not population:
        return pd.DataFrame()
    headers = ['District', 'Male', 'Female', 'Children', 'Total Population', 'Total Crop Area (in Hect.)', 'Population and Crop Area Details']
    df = pd.DataFrame(population[1:], columns=headers)
    return df

def extract_villages_affected(data):
    villages = data.get('Villages Affected', [])
    if not villages:
        return pd.DataFrame()
    headers = ['District', 'Total', 'Revenue Circle']
    df = pd.DataFrame(villages[1:], columns=headers)
    return df

def extract_animals_affected(data):
    animals = data.get('Animals Affected', [])
    if not animals:
        return pd.DataFrame()
    headers = ['District', 'Total', 'Big', 'Small', 'Poultry']
    df = pd.DataFrame(animals[1:], columns=headers)
    return df

In [26]:
def get_csv_files(directory):
    try:
        return [f for f in os.listdir(directory) if f.endswith('.csv') and f.startswith('DRIMS')]
    except FileNotFoundError:
        logging.error(f"Directory not found: {directory}")
        return []
    except PermissionError:
        logging.error(f"Permission denied to access directory: {directory}")
        return []
    except Exception as e:
        logging.error(f"Error accessing directory {directory}: {str(e)}")
        return []

In [27]:
def main(csv_directory, output_directory):
    csv_files = get_csv_files(csv_directory)

    if not csv_files:
        logging.warning(f"No matching CSV files found in the directory: {csv_directory}")
        return

    logging.info(f"Found {len(csv_files)} CSV files to process.")

    os.makedirs(output_directory, exist_ok=True)

    for csv_file in csv_files:
        full_path = os.path.join(csv_directory, csv_file)
        logging.info(f"Processing file: {full_path}")

        date = extract_date(csv_file)
        if not date:
            logging.warning(f"Could not extract date from filename: {csv_file}")
            continue

        data = process_csv(full_path)
        if data is None:
            logging.warning(f"Skipping file due to processing error: {csv_file}")
            continue

        extractions = {
            'BRIDGES_DAMAGED': extract_bridges_damaged,
            'EMBANKMENT_AFFECTED': extract_embankment_affected,
            'EMBANKMENT_BREACHED': extract_embankment_breached,
            'ROADS_DAMAGED': extract_roads_damaged,
            'HUMAN_LIFE_LOST': extract_human_life_lost,
            'POPULATION_AFFECTED': extract_population_affected,
            'VILLAGES_AFFECTED': extract_villages_affected,
            'ANIMALS_AFFECTED': extract_animals_affected
        }

        for name, extract_func in extractions.items():
            try:
                df = extract_func(data)
                if not df.empty:
                    output_file = os.path.join(output_directory, f"{name}_{date}.csv")
                    df.to_csv(output_file, index=False)
                    logging.info(f"Created {output_file}")
                else:
                    logging.warning(f"No data extracted for {name} from {csv_file}")
            except Exception as e:
                logging.error(f"Error extracting {name} from {csv_file}: {str(e)}")

        try:
            relief_camps = data.get('Relief Camps / Centres Opened', [])
            if relief_camps:
                headers = ['District', 'Total', 'Relief Camp', 'Relief Distribution Centres']
                df_relief_camps = pd.DataFrame(relief_camps[1:], columns=headers)
                output_file = os.path.join(output_directory, f"RELIEF_CAMPS_{date}.csv")
                df_relief_camps.to_csv(output_file, index=False)
                logging.info(f"Created {output_file}")
            else:
                logging.warning(f"No relief camps data found in {csv_file}")
        except Exception as e:
            logging.error(f"Error processing relief camps data from {csv_file}: {str(e)}")

In [28]:
if __name__ == "__main__":
    csv_directory = "/home/prajna/civicdatalab/IDS-DRR-Assam/Archive/Scrapers/FRIMS_Daily_Reports_Scraper/DRIMS_Reports_2024"
    output_directory = "/home/prajna/civicdatalab/IDS-DRR-Assam/Archive/Scrapers/FRIMS_Daily_Reports_Scraper/Data_2024/Scraped_Data"
    
    if not os.path.isdir(csv_directory):
        logging.error(f"The specified input directory does not exist: {csv_directory}")
        print(f"Error: The specified input directory does not exist: {csv_directory}")
    else:
        main(csv_directory, output_directory)

    logging.info("Script execution completed.")